In [3]:
import os
import json
import pickle
import numpy as np
import pandas as pd
from tqdm import tqdm
from scipy import stats
import matplotlib.pyplot as plt
from compute_tempo_aist import *
from aist_pos2s_script import *
# coco={    
# 0: "nose", 1: "left_eye", 2: "right_eye", 3: "left_ear",4: "right_ear",5: "left_shoulder",
# 6: "right_shoulder",7: "left_elbow",8: "right_elbow",9: "left_wrist",10: "right_wrist",
# 11: "left_hip",12: "right_hip",13: "left_knee",14: "right_knee",15: "left_ankle",16: "right_ankle",}  

# # cp ./video/gKR_sBM_c01_d28_mKR3_ch03.mp4 ./

In [2]:
def create_dir(tempo_dir):
    
    main_dir = "/itf-fi-ml/home/sagardu/aist_tempo_est/results/two_sensor"
    
    directories = [
        f"{tempo_dir}/pos/foot", f"{tempo_dir}/pos/hand", 
        f"{tempo_dir}/pos/score/foot", f"{tempo_dir}/pos/score/hand",
        f"{tempo_dir}/pos/tempo_data/ax0", f"{tempo_dir}/pos/tempo_data/ax1",
        
        f"{tempo_dir}/vel/foot", f"{tempo_dir}/vel/hand", 
        f"{tempo_dir}/vel/score/foot", f"{tempo_dir}/vel/score/hand",
        f"{tempo_dir}/vel/tempo_data/ax0", f"{tempo_dir}/vel/tempo_data/ax1",
        
        f"{tempo_dir}/stats_pos/uni", f"{tempo_dir}/stats_pos/bi",
        f"{tempo_dir}/stats_vel/uni", f"{tempo_dir}/stats_vel/bi",
        f"{tempo_dir}/stats_posvel/uni", f"{tempo_dir}/stats_posvel/bi"
    ]
    
    for dir_path in directories:
        full_path = os.path.join(main_dir, dir_path)
        os.makedirs(full_path, exist_ok=True)

In [7]:

marker_dict = {9: "left_wrist", 10: "right_wrist", 
                15: "left_ankle", 16: "right_ankle", 
                }   # 11: "left_hip",12: "right_hip"

config1 = {"sub_dir": ["hand"], "mode": ["zero_uni", "zero_bi"], 
           "markerA_id": 9, "markerB_id": 10, "a": 60, "b": 140, "metric": ["pos", "vel"]}
config2 = {"sub_dir": ["foot"], "mode": ["zero_uni", "zero_bi"], 
           "markerA_id": 15, "markerB_id": 16, "a": 60, "b": 140, "metric": ["pos", "vel"]}

configs = [config1, config2]

for cfg in configs:
    a = cfg["a"]
    b = cfg["b"]
    markerA_id = cfg["markerA_id"]
    markerB_id = cfg["markerB_id"]
    create_dir(f"tempo_{a}_{b}")
    
    for sub_dir in cfg["sub_dir"]:
        for mode in cfg["mode"]:
            for metric in cfg["metric"]:
                
                savepath = f"./results/two_sensor/tempo_{a}_{b}/{metric}/tempo_data"
                csv_filename = f"./results/two_sensor/tempo_{a}_{b}/{metric}/{sub_dir}/{sub_dir}_{mode}_{a}_{b}.csv"  
                
                aist_pos2s(a,b, mode, markerA_id, markerB_id, csv_filename, savepath,
                           w_sec = 5, vel_mode = "on" if metric == "vel" else "off")

100%|██████████| 1510/1510 [01:30<00:00, 16.67it/s]


Results saved to ./results/two_sensor/tempo_60_140/pos/hand/hand_zero_uni_60_140.csv


100%|██████████| 1510/1510 [01:36<00:00, 15.57it/s]


Results saved to ./results/two_sensor/tempo_60_140/vel/hand/hand_zero_uni_60_140.csv


100%|██████████| 1510/1510 [01:22<00:00, 18.31it/s]


...

Results saved to ./results/two_sensor/tempo_60_140/vel/foot/foot_zero_uni_60_140.csv


100%|██████████| 1510/1510 [01:12<00:00, 20.90it/s]


Results saved to ./results/two_sensor/tempo_60_140/pos/foot/foot_zero_bi_60_140.csv


100%|██████████| 1510/1510 [01:10<00:00, 21.29it/s]


Results saved to ./results/two_sensor/tempo_60_140/vel/foot/foot_zero_bi_60_140.csv


In [ ]:
result = {
    "filename": [],
    "dance_genre": [],
    "situation": [],
    "camera_id": [],
    "dancer_id": [],
    "music_id": [],
    "choreo_id": [],
    "music_tempo": [],

    "X_a": [],
    "Y_a": [],

    "bpm_mode": [],
    "bpm_median": [],
    "mode_x": [],
    "mode_y": [],

    "median_x": [],
    "median_y": [],
}

json_filename = "music_id_tempo.json"
with open(json_filename, "r") as file:
    aist_tempo = json.load(file)

f_path = "./aist_dataset/aist_annotation/keypoints2d"
aist_filelist = os.listdir(f_path)

mocap_fps = 60
a = 60; b =180
tempi_range = np.arange(a,b,1)   # good: 70,145 

marker_dict = {9: "left_wrist", 10: "right_wrist", 
               15: "left_ankle", 16: "right_ankle", 
               11: "left_hip",12: "right_hip"}
sub_dir = "hips" 
mode = "zero_bi" 
markerA_id = 11
markerB_id = 12
csv_filename = f"./results/aist_pos2s/tempo_{a}_{b}/{sub_dir}/{sub_dir}_{mode}_{a}_{b}.csv"        


for idx, filename in enumerate(tqdm(aist_filelist)):
    
    file_path = os.path.join(f_path, filename)
    file_info = filename.split("_")
    dance_genre = file_info[0] 
    situation = file_info[1] 
    camera_id = file_info[2] 
    dancer_id = file_info[3]
    music_id = file_info[4]
    choreo_id = file_info[5].strip(".pkl")
    
    with open(file_path, 'rb') as file:
        motion_data = pickle.load(file)

    markerA_x = motion_data["keypoints2d"][0, :, markerA_id, 0]     # array (n,)
    markerA_y = motion_data["keypoints2d"][0, :, markerA_id, 1]      # array (n,)
    
    markerB_x = motion_data["keypoints2d"][0, :, markerB_id, 0]     # array (n,)
    markerB_y = motion_data["keypoints2d"][0, :, markerB_id, 1]      # array (n,)
    

    if np.all((markerA_x == 0) & (markerA_y == 0)) or np.any(np.isnan(markerA_x) | np.isnan(markerA_y)):
        continue
    
    markerA_x = detrend_signal_array(markerA_x.reshape(-1, 1), cutoff= 1, fs=60)
    markerA_y = detrend_signal_array(markerA_y.reshape(-1, 1), cutoff= 1, fs=60)
    markerA_pos = np.concatenate((markerA_x, markerA_y), axis=1)  # size (n,2)
    
    markerB_x = detrend_signal_array(markerB_x.reshape(-1, 1), cutoff= 1, fs=60)
    markerB_y = detrend_signal_array(markerB_y.reshape(-1, 1), cutoff= 1, fs=60)
    markerB_pos = np.concatenate((markerB_x, markerB_y), axis=1)  # size (n,2)
    
    
    
    duration = int(len(markerA_x)/60)
    w_sec = int(duration)
    h_sec = int(w_sec/4)
    window_size = int(mocap_fps*w_sec)
    hop_size = int(mocap_fps*h_sec)
    
    bpm_axes = []
    for ax in range(2):
    
        posA_min, posA_max = np.min(markerA_pos[:, ax]), np.max(markerA_pos[:, ax])
        sensorA_position_norm = (
                2*(markerA_pos[:, ax] - posA_min) / (posA_max - posA_min) - 1
                if posA_max != posA_min 
                else np.zeros_like(markerA_pos[:, ax])
            )

        posB_min, posB_max = np.min(markerB_pos[:, ax]), np.max(markerB_pos[:, ax])
        sensorB_position_norm = (
                2*(markerB_pos[:, ax] - posB_min) / (posB_max - posB_min) - 1
                if posB_max != posB_min
                else np.zeros_like(markerB_pos[:, ax])
            )
        
        
        tempo_json = main_two_sensor(sensorA_position_norm.reshape(-1,1), sensorB_position_norm.reshape(-1,1),  
                                    mocap_fps, window_size, hop_size, tempi_range,
                                    T_filter= 0.25, smooth_wlen= 10, pk_order = 15, mode=mode)
    
        tempo_data_maxmethod = tempo_json["tempo_data_maxmethod"]
        bpmA_arr = tempo_data_maxmethod["bpm_arr"]
        tempo_A = np.round(np.average(bpmA_arr), 2)
        bpm_axes.append(bpmA_arr)
        
        mode_x = stats.mode(bpmA_arr.flatten())[0]
        mode_y = stats.mode(bpmA_arr.flatten())[0]

        median_x = np.median(bpmA_arr.flatten())
        median_y = np.median(bpmA_arr.flatten())

        
        if ax == 0:
            result["filename"].append(filename.strip(".pkl"))
            result["dance_genre"].append(dance_genre)
            result["situation"].append(situation)
            result["camera_id"].append(camera_id)
            result["dancer_id"].append(dancer_id)
            result["music_id"].append(music_id)
            result["choreo_id"].append(choreo_id)
            result["music_tempo"].append(aist_tempo[music_id])
            
            result["mode_x"].append(mode_x)
            result["mode_y"].append(mode_y)

            
            result["median_x"].append(median_x)
            result["median_y"].append(median_y)

            
            result["X_a"].append(tempo_A)

        elif ax == 1:
            result["Y_a"].append(tempo_A)

    bpm_axes_arr = np.column_stack(bpm_axes)    # n by 3 array
    bpm_mode = stats.mode(bpm_axes_arr.flatten())[0]
    bpm_median = np.median(bpm_axes_arr.flatten())
    result["bpm_mode"].append(bpm_mode)
    result["bpm_median"].append(bpm_median)    
    
    # if idx == 6:
    #     break    
    
results_df = pd.DataFrame(result)

results_df.to_csv(csv_filename, index=False)
print(f"Results saved to {csv_filename}")         

    



In [ ]:
# Extract x and y positions for the given keypoint
markerA_x = motion_data["keypoints2d"][0, :, 10, 0]
markerA_y = motion_data["keypoints2d"][0, :, 10, 1]

# Normalize x and y to [0,1] range
x_min, x_max = np.min(markerA_x), np.max(markerA_x)
y_min, y_max = np.min(markerA_y), np.max(markerA_y)

x_norm = (markerA_x - x_min) / (x_max - x_min) if x_max != x_min else np.zeros_like(markerA_x)
y_norm = (markerA_y - y_min) / (y_max - y_min) if y_max != y_min else np.zeros_like(markerA_y)

# Smooth the normalized position data
xs = smooth_velocity(x_norm.reshape(-1, 1), abs="no", window_length=10, polyorder=0)
ys = smooth_velocity(y_norm.reshape(-1, 1), abs="no", window_length=10, polyorder=0)


# Plot position and velocity
fig, axs = plt.subplots(1, 2, figsize=(10, 4), dpi=100)
fig.suptitle('Position and Velocity')

# Position plots
axs[0].plot(xs, label="x position")
axs[1].plot(ys, label="y position")
axs[0].set_title("X Position")
axs[1].set_title("Y Position")

# Formatting
for ax in axs.flat:
    ax.legend()
    ax.grid(True)

plt.tight_layout()
plt.show()
